#Setup Colab

In [1]:
from google.colab import drive 

drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down/"
!pwd
!ls -l

# Install if you haven't done so.
!pip install tensorflow==2.2.0
!pip install ray[rllib]==0.8.6   
#!pip show tensorflow
#!pip show ray

!rm -rf ~/ray_results/DDPPO_*
!rm -rf ~/ray_results/PPO_*

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down
/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down
total 158
drwx------ 6 root root   4096 Aug  4 07:50 chkpt
-rw------- 1 root root   3139 Jun 11 03:59 Helper.py
-rw------- 1 root root      1 Jun 11 03:59 __init__.py
-rw------- 1 root root   1072 Jun 10 04:55 LICENSE
-rw------- 1 root root   6021 Aug  4 04:44 PBT_MARL_old.py
-rw------- 1 root root   6258 Aug  4 06:55 PBT_MARL.py
-rw------- 1 root root   9014 Jun 10 17:23 pbt_marl_water_down_cpu_only.py
-rw------- 1 root root 115164 Aug  4 08:00 PBT_MARL_water_down.ipynb
drwx------ 2 root root   4096 Jul 31 11:40 __pycache__
drwx------ 2 root root   4096 Aug  4 07:58 ray_results
-rw------- 1 root root   3705 Jul 28 05:35 README.md
-rw------- 1 root root   2056 Aug  4 06:41 RockPaperScissorsEnv.py


#Chkpt/restore & log path

In [2]:
g_drive_path = "/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down/"

local_dir = g_drive_path + "chkpt/"
chkpt_freq = 10
chkpt = 150
restore_path = "{}checkpoint_{}/checkpoint-{}".format(local_dir, chkpt, chkpt)
is_restore = False

log_dir = g_drive_path + "ray_results/"

#Imports

In [3]:
from collections import defaultdict
from typing import Dict
import random
import numpy as np

from gym.spaces import Discrete

import ray
from ray import tune

from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog

from ray.rllib.policy import Policy

from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.ppo import appo
from ray.rllib.agents.ppo.appo import APPOTrainer
from ray.rllib.agents.ppo import ddppo
from ray.rllib.agents.ppo.ddppo import DDPPOTrainer

from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv

from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.agents.callbacks import DefaultCallbacks

from ray.rllib.utils.schedules import ConstantSchedule
from ray.rllib.utils import try_import_tf
from ray.tune.logger import pretty_print

tf = try_import_tf()

from RockPaperScissorsEnv import RockPaperScissorsEnv
from Helper import Helper
from PBT_MARL import PBT_MARL

#Callbacks

In [4]:
"""#Callbacks"""

class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, **kwargs):
        #print("on_episode_start {}, _agent_to_policy {}".format(episode.episode_id, episode._agent_to_policy))
        #episode.hist_data["episode_id"] = []
        pass

    def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
                        episode: MultiAgentEpisode, **kwargs):
        """
        pole_angle = abs(episode.last_observation_for()[2])
        raw_angle = abs(episode.last_raw_obs_for()[2])
        assert pole_angle == raw_angle
        episode.user_data["pole_angles"].append(pole_angle)
        """
        pass

    def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
        #print("on_episode_end {}, episode.agent_rewards {}".format(episode.episode_id, episode.agent_rewards))

        player_policy = []
        score = []
        for k,v in episode.agent_rewards.items():
            player_policy.append(k)
            score.append(v)

        pol_i_key = player_policy[0][1]
        pol_j_key = player_policy[1][1]
        _, str_i = pol_i_key.split("_")
        _, str_j = pol_j_key.split("_")
        agt_i_key = "agt_" + str_i
        agt_j_key = "agt_" + str_j

        g_helper = ray.get_actor("g_helper")     
        prev_rating_i = ray.get(g_helper.get_rating.remote(agt_i_key))
        prev_rating_j = ray.get(g_helper.get_rating.remote(agt_j_key))
        score_i = score[0]
        score_j = score[1]
        rating_i, rating_j = l_PBT_MARL.compute_rating(prev_rating_i, prev_rating_j, score_i, score_j)
        ray.get(g_helper.update_rating.remote(agt_i_key, agt_j_key, rating_i, rating_j, score_i, score_j))
        #print("on_episode_end ray.get(g_helper.get_agt_store.remote())", ray.get(g_helper.get_agt_store.remote()))

    def on_sample_end(self, worker: RolloutWorker, samples: SampleBatch,
                      **kwargs):
        #print("on_sample_end returned sample batch of size {}".format(samples.count))
        pass

    def on_train_result(self, trainer, result: dict, **kwargs):
        print("trainer.train() result: {} -> {} episodes".format(trainer, result["episodes_this_iter"]))
        # you can mutate the result dict to add new fields to return
        result["callback_ok"] = True
        #print("on_train_result result", result)

        l_PBT_MARL.PBT(trainer)     # perform PBT

        g_helper = ray.get_actor("g_helper")     
        ray.get(g_helper.set_pair.remote())     # set the lastest pair
        #print("on_train_result g_helper.get_pair.remote()", ray.get(g_helper.get_pair.remote()))

    def on_postprocess_trajectory(
            self, worker: RolloutWorker, episode: MultiAgentEpisode,
            agent_id: str, policy_id: str, policies: Dict[str, Policy],
            postprocessed_batch: SampleBatch,
            original_batches: Dict[str, SampleBatch], **kwargs):
        #print("postprocessed {}, {}, {}, {} steps".format(episode, agent_id, policy_id, postprocessed_batch.count))              
        """
        if "num_batches" not in episode.custom_metrics:
            episode.custom_metrics["num_batches"] = 0
        episode.custom_metrics["num_batches"] += 1
        """
        pass

#Policy

In [5]:
def init_policies(population_size, obs_space, act_space, use_lstm, hyperparameters_range):
    """
    Sample hyper-parameter from the hyper-parameter distribution.
    """
    policies = {}
    for i in range(population_size):
        pol_key = "p_" + str(i)
        lr = np.random.uniform(low=hyperparameters_range["lr"][0], high=hyperparameters_range["lr"][1], size=None)
        gamma = np.random.uniform(low=hyperparameters_range["gamma"][0], high=hyperparameters_range["gamma"][1], size=None)
        policies[pol_key] = (None, obs_space, act_space, {"model": {"use_lstm": use_lstm},
                                                          "lr": lr,
                                                          "gamma": gamma})
    return policies

def train_policies(population_size):    
    train_policies = []
    for i in range(population_size):
        pol_key = "p_" + str(i)
        train_policies.append(pol_key)

    return policies

def select_policy(agent_id):
    _, i = agent_id.split("_")
    policy = "p_" + str(i)
    #print("select_policy {} {}".format(agent_id , policy))
    return policy     

#Variables

In [6]:
population_size = 6
K = 0.1     
T_select = 0.77 #0.47
binomial_n = 1
inherit_prob = 0.5
perturb_prob = 0.1
perturb_val = [0.8, 1.2]
hyperparameters_range = {"lr": [1, 1], 
                         "gamma": [0.9, 0.999]}

register_env("RockPaperScissorsEnv", lambda _: RockPaperScissorsEnv(_, population_size))     # register RockPaperScissorsEnv with RLlib     
# get obs & act spaces from dummy CDA env
dummy_env = RockPaperScissorsEnv(_, population_size=0)
obs_space = dummy_env.observation_space
act_space = dummy_env.action_space

use_lstm=False
policies = init_policies(population_size, obs_space, act_space, use_lstm, hyperparameters_range)
train_policies = train_policies(population_size)

l_PBT_MARL = PBT_MARL(population_size, 
                      K, T_select, 
                      binomial_n, inherit_prob,
                      perturb_prob, perturb_val)

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=True, webui_host='127.0.0.1', num_cpus=2, num_gpus=1)      #start ray
#print("ray.nodes()", ray.nodes())

g_helper = Helper.options(name="g_helper").remote(population_size, policies)      # this object runs on a different ray actor process
ray.get(g_helper.set_pair.remote())

num_iters = 30     # num of main training loop

2020-08-04 08:01:06,156	INFO resource_spec.py:212 -- Starting Ray with 7.28 GiB memory available for workers and up to 3.64 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-04 08:01:06,410	WARNING services.py:923 -- Redis failed to start, retrying now.
2020-08-04 08:01:06,764	INFO services.py:1165 -- View the Ray dashboard at 127.0.0.1:8265


#Config

In [7]:
def get_config():
    #config = ddppo.DEFAULT_CONFIG.copy()
    config = ppo.DEFAULT_CONFIG.copy()

    config["env"] = RockPaperScissorsEnv
    config["multiagent"] = {"policies_to_train": train_policies,
                            "policies": policies,
                            "policy_mapping_fn": select_policy}        
    config["num_cpus_per_worker"] = 0.25                                
    config["num_gpus_per_worker"] = 0.125
    config["num_workers"] = 2      
    config["num_envs_per_worker"] = 3
    config["rollout_fragment_length"] = 30                  
    #config["train_batch_size"] = -1     # must be -1 for DDPPO trainer 
    config["train_batch_size"] = 90                                                     
    config["sgd_minibatch_size"] = 10                       
    config["num_sgd_iter"] = 3      # number of epochs to execute per train batch.
    config["callbacks"] = MyCallbacks
    config["log_level"] = "WARN"      # WARN/INFO/DEBUG 
    config["output"] = log_dir

    return config

#Go train

In [8]:
def go_train(config):     
    #trainer = ddppo.DDPPOTrainer(config=config, env="RockPaperScissorsEnv")         
    trainer = ppo.PPOTrainer(config=config, env="RockPaperScissorsEnv")         

    if is_restore == True:
        trainer.restore(restore_path) 
    
    result = None
    for i in range(num_iters):
        result = trainer.train()       
        print("training loop = {} of {}".format(i + 1, num_iters))            
        print(pretty_print(result))     # includes result["custom_metrics"]

        #p_0 = trainer.get_policy('p_0')
        #p_0.lr_schedule = ConstantSchedule(0.3, framework=None)

        if i % chkpt_freq == 0:
            checkpoint = trainer.save(local_dir)
            print("checkpoint saved at", checkpoint)
    
    checkpoint = trainer.save(local_dir)
    print("checkpoint saved at", checkpoint)
    

# run everything
go_train(get_config())    

ray.shutdown()

2020-08-04 08:01:10,348	INFO trainer.py:585 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2020-08-04 08:01:10,354	INFO trainer.py:612 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-08-04 08:02:13,892	INFO trainable.py:181 -- _setup took 63.548 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2020-08-04 08:02:13,894	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-08-04 08:02:21,517	WARNING ppo.py:166 -- No data for p_0, not updating kl
2020-08-04 08:02:21,819	WARNING ppo.py:166 -- No data for p_2, not updating kl
2020-08-04 08:02:21,820	WARNING ppo.py:166 -- No data for p_3, not updating kl
2020-08-04 08:02:22,126	WARNING ppo.py:166 -- No data for p_5, not updating kl


trainer.train() result: <ray.rllib.agents.trainer_template.PPO object at 0x7fe2d6a6b550> -> 18 episodes
training loop = 1 of 30
callback_ok: true
custom_metrics: {}
date: 2020-08-04_08-02-22
done: false
episode_len_mean: 10.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 18
episodes_total: 18
experiment_id: 143681295c0a4ea89f1fc8c8dfcce63f
hostname: 34fd3023fdc6
info:
  learner:
    p_1:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 1.0
      entropy: 0.04228334128856659
      entropy_coeff: 0.0
      kl: 50.581275939941406
      model: {}
      policy_loss: 0.40675604343414307
      total_loss: 150.04600524902344
      vf_explained_var: -0.6395921111106873
      vf_loss: 139.52297973632812
    p_4:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 1.0
      entropy: 0.01735462248325348
      entropy_coeff: 0.0
      kl: 51.05513000488281
      model: {}
      policy_loss: 0.4643787741661072
      total_loss: 398.4315185546875


2020-08-04 08:02:28,040	WARNING ppo.py:166 -- No data for p_0, not updating kl
2020-08-04 08:02:28,045	WARNING ppo.py:166 -- No data for p_2, not updating kl
2020-08-04 08:02:28,051	WARNING ppo.py:166 -- No data for p_3, not updating kl


trainer.train() result: <ray.rllib.agents.trainer_template.PPO object at 0x7fe2d6a6b550> -> 18 episodes
training loop = 2 of 30
callback_ok: true
custom_metrics: {}
date: 2020-08-04_08-02-28
done: false
episode_len_mean: 10.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 18
episodes_total: 36
experiment_id: 143681295c0a4ea89f1fc8c8dfcce63f
hostname: 34fd3023fdc6
info:
  learner:
    p_1:
      cur_kl_coeff: 0.30000001192092896
      cur_lr: 1.0
      entropy: 1.971934552343906e-37
      entropy_coeff: 0.0
      kl: 6.29649166139501e-10
      model: {}
      policy_loss: -5.612770905827347e-07
      total_loss: 3.965785264968872
      vf_explained_var: 0.21986787021160126
      vf_loss: 3.9657859802246094
    p_4:
      cur_kl_coeff: 0.30000001192092896
      cur_lr: 1.0
      entropy: 0.0
      entropy_coeff: 0.0
      kl: 1.4843363302642154e-22
      model: {}
      policy_loss: -1.0761949553739214e-08
      total_loss: 55.918888092041016

2020-08-04 08:02:30,533	WARNING ppo.py:166 -- No data for p_0, not updating kl
2020-08-04 08:02:30,535	WARNING ppo.py:166 -- No data for p_1, not updating kl
2020-08-04 08:02:30,540	WARNING ppo.py:166 -- No data for p_2, not updating kl


...

trainer.train() result: <ray.rllib.agents.trainer_template.PPO object at 0x7fe2d6a6b550> -> 18 episodes
training loop = 28 of 30
callback_ok: true
custom_metrics: {}
date: 2020-08-04_08-03-04
done: false
episode_len_mean: 10.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 18
episodes_total: 504
experiment_id: 143681295c0a4ea89f1fc8c8dfcce63f
hostname: 34fd3023fdc6
info:
  learner:
    p_0:
      cur_kl_coeff: 0.00010986327833961695
      cur_lr: 0.015949947759509087
      entropy: 1.6564308258359972e-10
      entropy_coeff: 0.0
      kl: 1.2980590393050205e-15
      model: {}
      policy_loss: 3.0914734661102727e-10
      total_loss: 3.4795244463481367e-10
      vf_explained_var: -1.0
      vf_loss: 6.803967805835143e-11
    p_2:
      cur_kl_coeff: 0.0009887695778161287
      cur_lr: 0.012759958393871784
      entropy: 1.6564309646138753e-10
      entropy_coeff: 0.0
      kl: 1.2980590393050205e-15
      model: {}
      policy_loss: 0.0


2020-08-04 08:03:05,956	WARNING ppo.py:166 -- No data for p_2, not updating kl
2020-08-04 08:03:05,962	WARNING ppo.py:166 -- No data for p_4, not updating kl
2020-08-04 08:03:05,963	WARNING ppo.py:166 -- No data for p_5, not updating kl


trainer.train() result: <ray.rllib.agents.trainer_template.PPO object at 0x7fe2d6a6b550> -> 18 episodes
training loop = 29 of 30
callback_ok: true
custom_metrics: {}
date: 2020-08-04_08-03-05
done: false
episode_len_mean: 10.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 18
episodes_total: 522
experiment_id: 143681295c0a4ea89f1fc8c8dfcce63f
hostname: 34fd3023fdc6
info:
  learner:
    p_0:
      cur_kl_coeff: 5.493163916980848e-05
      cur_lr: 0.012759958393871784
      entropy: 1.6564309646138753e-10
      entropy_coeff: 0.0
      kl: 1.2980590393050205e-15
      model: {}
      policy_loss: -9.313225746154785e-10
      total_loss: 6.208817349140361e-10
      vf_explained_var: -1.0
      vf_loss: 1.6975367778115924e-09
    p_1:
      cur_kl_coeff: 0.0004394531133584678
      cur_lr: 0.010207965970039368
      entropy: 1.6564309646138753e-10
      entropy_coeff: 0.0
      kl: 1.2980590393050205e-15
      model: {}
      policy_loss: 3.750

2020-08-04 08:03:07,114	WARNING ppo.py:166 -- No data for p_0, not updating kl
2020-08-04 08:03:07,124	WARNING ppo.py:166 -- No data for p_4, not updating kl
2020-08-04 08:03:07,130	WARNING ppo.py:166 -- No data for p_5, not updating kl


trainer.train() result: <ray.rllib.agents.trainer_template.PPO object at 0x7fe2d6a6b550> -> 18 episodes
training loop = 30 of 30
callback_ok: true
custom_metrics: {}
date: 2020-08-04_08-03-07
done: false
episode_len_mean: 10.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 18
episodes_total: 540
experiment_id: 143681295c0a4ea89f1fc8c8dfcce63f
hostname: 34fd3023fdc6
info:
  learner:
    p_1:
      cur_kl_coeff: 0.0002197265566792339
      cur_lr: 0.012249560095369816
      entropy: 1.6564308258359972e-10
      entropy_coeff: 0.0
      kl: 1.2980590393050205e-15
      model: {}
      policy_loss: 2.1523899107478428e-08
      total_loss: 2.1523899107478428e-08
      vf_explained_var: -1.0
      vf_loss: 6.314014200370366e-09
    p_2:
      cur_kl_coeff: 0.0004943847889080644
      cur_lr: 0.018374338746070862
      entropy: 1.6564309646138753e-10
      entropy_coeff: 0.0
      kl: 1.2980590393050205e-15
      model: {}
      policy_loss: -2.48